# Format data
Dans ce fichier, on importe les differentes sources, et on les normalise et formatte pour utiliser un meme format

## Intro: installation!

In [11]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [12]:
import pandas as pd
import re

## Fichier corpus

In [13]:
# Inspire par: http://abdulbaqi.io/2019/01/19/quranic-roots-pandas/

# Lire le fichier de quranic corpus (en sautant les lignes d'intro)
corpus = pd.read_csv('./original_data/quranic-corpus-morphology-0.4.txt', sep='\t',skiprows=56)



# L'enregistrer converti en csv
corpus.to_csv('./original_data/quranic-corpus-morphology.csv')



In [14]:
# Voir un sample 
corpus.head()

,LOCATION,FORM,TAG,FEATURES
0,(1:1:1:1),bi,P,PREFIX|bi+
1,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN
2,(1:1:2:1),{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN
3,(1:1:3:1),{l,DET,PREFIX|Al+
4,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN


In [15]:
# Compter lignes
corpus.count()

LOCATION    128219
FORM        128011
TAG         128219
FEATURES    128219
dtype: int64

In [16]:
# Ajouter une colonne pour "root"


# Function to extract the value of 'root' from 'features' using regex
def extraire_root(features):
    match = re.search(r"\|ROOT:([^|]+)", features)
    if match:
        return match.group(1)
    else:
        return None

# Apply the 'extract_root' function to create the 'root' column
corpus['root'] = corpus['FEATURES'].apply(extraire_root)


corpus.head()

,LOCATION,FORM,TAG,FEATURES,root
0,(1:1:1:1),bi,P,PREFIX|bi+,None
1,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN,smw
2,(1:1:2:1),{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN,Alh
3,(1:1:3:1),{l,DET,PREFIX|Al+,None
4,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN,rHm


In [17]:
# Fonction pour extraire la position du mot au niveau 3 (par exemple 6:11:2)
def extraire_position_mot(location):
    match = re.match(r"\((\d+:\d+:\d+)", location)
    if match:
        return match.group(1)
    else:
        return None

# Appliquer la fonction pour creer "word_location"
corpus['word_location'] = corpus['LOCATION'].apply(extraire_position_mot)

corpus.head()

,LOCATION,FORM,TAG,FEATURES,root,word_location
0,(1:1:1:1),bi,P,PREFIX|bi+,None,1:1:1
1,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN,smw,1:1:1
2,(1:1:2:1),{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN,Alh,1:1:2
3,(1:1:3:1),{l,DET,PREFIX|Al+,None,1:1:3
4,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN,rHm,1:1:3


In [29]:
# Extraire le verset de 'LOCATION'
def extraire_verset(location):
    match = re.match(r"\((\d+:\d++)", location)
    if match:
        return match.group(1)
    else:
        return None

# Apply the 'extraire_verset' function to create the 'verse' column
corpus['verse'] = corpus['LOCATION'].apply(extraire_verset)


corpus.head()

,LOCATION,FORM,TAG,FEATURES,root,word_location,verse
0,(1:1:1:1),bi,P,PREFIX|bi+,None,1:1:1,1:1
1,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN,smw,1:1:1,1:1
2,(1:1:2:1),{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN,Alh,1:1:2,1:1
3,(1:1:3:1),{l,DET,PREFIX|Al+,None,1:1:3,1:1
4,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN,rHm,1:1:3,1:1


In [19]:
# compter le nombre de mots selon cette source
corpus['word_location'].nunique()

77429

-## Fichier Coran mot a mot

In [21]:
coran_mot = pd.read_csv('./original_data/Coran_mot_a_mot_coranseul_12juin2019.csv')
coran_mot.head()

,N° de mot,Mot arabe,Translitération,Racine,Traduction
0,1:1#1,بِسْمِ,biçmi,سمو,Par le biais de l'attribut de
1,1:1#2,ٱللَّهِ,allahi,الله,Dieu
2,1:1#3,ٱلرَّحْمَٰنِ,alraĥmani,رحم,"le Très Bienveillant,"
3,1:1#4,ٱلرَّحِيمِ,alraĥiymi,رحم,le vecteur de toute Bienveillance (Bienveilleur).
4,1:2#1,ٱلْحَمْدُ,alĥamdou,حمد,Louanges


In [22]:
# compter le nombre de mots selon cette source
coran_mot['N° de mot'].nunique()

77399

In [23]:
# Il y a une disparite. On va maintenant trouver ou. 
# Je vais creer la colonne word_location sur coran_mot aussi avec le meme format que dans l'autre.
# Formatter 'N° de mot' en format 1:1:1
def formatter_numero_en_word_location(numero):
    return numero.replace("#", ":")

coran_mot['word_location'] = coran_mot['N° de mot'].apply(formatter_numero_en_word_location)

coran_mot.head()

,N° de mot,Mot arabe,Translitération,Racine,Traduction,word_location
0,1:1#1,بِسْمِ,biçmi,سمو,Par le biais de l'attribut de,1:1:1
1,1:1#2,ٱللَّهِ,allahi,الله,Dieu,1:1:2
2,1:1#3,ٱلرَّحْمَٰنِ,alraĥmani,رحم,"le Très Bienveillant,",1:1:3
3,1:1#4,ٱلرَّحِيمِ,alraĥiymi,رحم,le vecteur de toute Bienveillance (Bienveilleur).,1:1:4
4,1:2#1,ٱلْحَمْدُ,alĥamdou,حمد,Louanges,1:2:1


In [24]:
# Trouver les word_location qui sont dans une source et pas l'autre


# Un set recupere les valeurs uniques
corpus_word_locations = set(corpus['word_location'])
coran_mot_word_locations = set(coran_mot['word_location'])

# les word_location dans corpus qui ne sont pas dans coran_mot
seulement_dans_corpus = corpus_word_locations - coran_mot_word_locations

# Find the word_location values that appear in df2 but not in df1
seulement_dans_coran_mot = coran_mot_word_locations - corpus_word_locations

# Print the results
print("Dans corpus mais pas coran_mot:", seulement_dans_corpus)
print("Dans coran_mot mais pas corpus:", seulement_dans_coran_mot)

Dans corpus mais pas coran_mot: {'9:129:12', '9:128:9', '9:129:15', '9:128:8', '9:129:9', '9:128:11', '9:129:8', '96:5:5', '9:128:2', '9:128:5', '9:128:1', '9:128:10', '9:128:3', '9:129:14', '9:129:1', '9:129:2', '9:129:3', '9:129:7', '9:129:10', '9:128:6', '9:128:13', '9:129:5', '9:128:14', '9:128:12', '9:129:13', '9:129:4', '9:129:6', '9:129:11', '9:128:4', '9:128:7'}
Dans coran_mot mais pas corpus: set()


In [25]:
# En gros, il manque les versets 9:128 et 9:129 dans Coran mot a mot. Il manque aussi le mot 96:5:5.
# Hypothese: il y a un decalage? Comparons le verset precedent, 9:127.

# d'abord je cree une versino de corpus qui aggregue les FORM au niveau de word_location. 

corpus_word = corpus.groupby('word_location')['FORM'].apply(lambda x: ' '.join(map(str, x))).reset_index(name='transliteration_corpus')

corpus_word.head()

,word_location,transliteration_corpus
0,100:10:1,wa HuS~ila
1,100:10:2,maA
2,100:10:3,fiY
3,100:10:4,{l S~uduwri
4,100:11:1,<in~a


In [26]:
# Ensuite je compare les deux au verset 9:127
joined = corpus_word.merge(coran_mot, on="word_location", how="outer")

joined_9_127 = joined.query('word_location.str.startswith("9:127")')
joined_9_127.head()
# Ca m'a l'air correct. Donc ca veut dire qu'il manque 9:128 et 9:129 dans coran mot a mot. 
# Je ne comprends pas trop pourquoi ces deux versets manquent.

,word_location,transliteration_corpus,N° de mot,Mot arabe,Translitération,Racine,Traduction
75517,9:127:1,wa <i*aA,9:127#1,وَإِذَا,wa'îđa,إذا,et lorsque
75518,9:127:10,yaraY` kum,9:127#10,يَرَىٰكُم,yarakoum,رأي,il vous perçoit
75519,9:127:11,m~ino,9:127#11,مِّنْ,min,أم,provenant de / qui / contre / parmi
75520,9:127:12,>aHadK,9:127#12,أَحَدٍ,aĥadin,أحد,un seul / quelqu'un
75521,9:127:13,vum~a,9:127#13,ثُمَّ,thouma,ثم,ensuite


In [28]:

# Par contre pourquoi le mot 96:5:5, c'est surement plus anodin. Regardons:

joined_96_5 = joined.query('word_location.str.startswith("96:5")')
joined_96_5.head()

# Effectivement cette difference m'a l'air anodine! 
# Corpus considere que lamo est un mot separe et coran mot a mot non.

,word_location,transliteration_corpus,N° de mot,Mot arabe,Translitération,Racine,Traduction
74752,96:5:1,Eal~ama,96:5#1,عَلَّمَ,Ɛalama,علم,Il enseigne à
74753,96:5:2,{lo <insa`na,96:5#2,ٱلْإِنسَٰنَ,al'înçana,أنس,l'être humain
74754,96:5:3,maA,96:5#3,مَالَمْ,malam,ما,ce que jamais
74755,96:5:4,lamo,96:5#4,يَعْلَمْ,yaƐlam,علم,il (n’)a su.
74756,96:5:5,yaEolamo,NaN,NaN,NaN,NaN,NaN
